In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import numpy as np
from handle_dataset.transform import create_df_with_datetimes

In [2]:
train_df = pd.read_csv('Dataset/Yearly-train.csv')
test_df = pd.read_csv('Dataset/Yearly-test.csv')

In [3]:

counter = 0

train = create_df_with_datetimes(train_df, counter)
test = create_df_with_datetimes(test_df, counter)
test.columns = ['ds','y']
train.columns= ['ds','y']
# Instantiate and fit the model
model = Prophet(interval_width=0.95)
model.fit(train)

/home/george/.local/lib/python3.8/site-packages/prophet/forecaster.py:1124: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.history_dates = pd.to_datetime(pd.Series(df['ds'].unique(), name='ds')).sort_values()
/home/george/.local/lib/python3.8/site-packages/prophet/forecaster.py:271: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ds'] = pd.to_datetime(df['ds'])
20:44:41 - cmdstanpy - INFO - Chain [1] start processing
20:44:41 - cmdstanpy - INFO - Chain [1] done processing


In [4]:
future_dates = model.make_future_dataframe(periods=6, freq='YS')
future_dates.head()

,ds
0,1980-01-01 18:00:00
1,1981-01-01 00:00:00
2,1982-01-01 06:00:00
3,1983-01-01 12:00:00
4,1984-01-01 18:00:00


In [5]:
# Create a dataframe with future dates for prediction
# future = model.make_future_dataframe(periods=30)  # Forecasting for an additional 30 days

# Generate forecasts
forecast = model.predict(future_dates)

forecast_organized = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
# print(forecast_organized)

# fig1 = model.plot(forecast_organized)

# Plot the forecast
# fig = model.plot(forecast)
# plt.title('Prophet Forecasting')
# plt.show()

In [8]:
forecast_organized.head()

,ds,yhat,yhat_lower,yhat_upper
0,1980-01-01 18:00:00,4955.595810,4736.539365,5182.301774
1,1981-01-01 00:00:00,5069.599119,4835.448712,5289.841190
2,1982-01-01 06:00:00,5183.602429,4962.067623,5434.183633
3,1983-01-01 12:00:00,5297.605739,5076.119889,5529.548098
4,1984-01-01 18:00:00,5411.626129,5185.836513,5629.597853


In [13]:
forecast_organized['yhat'].size

37

In [15]:
train['y'].size

31

In [16]:
test['y'].size

6

In [19]:
forecast['yhat'][-6:]

31    7606.798283
32    7635.551155
33    7674.697970
34    7710.318919
35    7742.504784
36    7771.257656
Name: yhat, dtype: float64

In [20]:
test['y']

1    7290.2
2    7392.6
3    7527.6
4    7594.8
5    7720.7
6    7823.2
Name: y, dtype: object

In [22]:
from evaluation_protocol.grubbs import grubbs_score
from evaluation_protocol.mape import mape
from evaluation_protocol.smape import smape
from evaluation_protocol.shape_similarity import dtw

In [23]:
alpha = 0.05
predicted = forecast['yhat'][-6:]
real = test['y']
grubbs_test_score = grubbs_score(predicted, real, alpha)
smape_score = smape(real.tolist(), predicted.tolist())
shape_similarity_score = dtw(predicted, real)
mape_score = mape(real.tolist(), predicted.tolist())

model_name = 'prophet'

print(f"Grubbs score for {model} is : {grubbs_test_score}")
print(f"SMAPE score for {model_name} is : {smape_score}")
print(f"MAPE score for {model_name} is : {mape_score}")
print(f"Shape similarity score for {model_name} is : {shape_similarity_score}")
print("===================================================================================")

No outliers detected.
No outliers detected.
Grubbs score for <prophet.forecaster.Prophet object at 0x7ff7d993cc10> is : 0
SMAPE score for prophet is : 1.9794528840151553
MAPE score for prophet is : 2.0084528351210076
Shape similarity score for prophet is : 780.8762437136947
